# TetCTF 2020 - Writeups for Crypto challenges

-   By: [ndh](https://twitter.com/_ndh___)

## 2019rearrange
---


### Challenge summary
Find $m$, given $n, e_1, e_2, a_1, a_2, c_1 = (m + a_1)^{e_1} \text{ mod } n, c_2 = (m + a_2)^{e_2} \text{ mod } n$.



### Solution
Let $f_1, f_2 \in Z_n[X]$ and $f_1 = (X + a_1)^{e_1} - c_1, f_2 = (X + a_2)^{e_2} - c_2$. Since $f_1(m) = f_2(m) = 0$, they're both divisible by $X-m$. Computing $\text{GCD}(f_1, f_2)$ should give us $X-m$. The challenge is now solved.

## 2020th
---


### Challenge summary
Let $(r_1, r_2, r_3, ... r_{2020})$ be a sequence of 2020 consecutive 32-bit integers output by the Mersenne twister used by the random module of Python 3. We're allowed to know 2 numbers in the sequence (except $r_{2020}$) of our choice, and have to make a guess for $r_{2020}$.



### Solution
The internal state of the Mersenne twister can be considered as a stream of 32-bit integers $(s_i)$, $s_{i+624} = g(s_i, s_{i+1}, s_{i+397})$; and the pseudo-random output is also another stream of 32-bit integers: $(r_i)$, $r_i = f(s_i)$ as in the figure below:

![](mersenne-twister.png)

Please also note that the function $f$ is invertible and $g$ only uses the most significant bit of $s_i$.

To solve the challenge, we need to:
1.  Ask for $r_{1397}$ ($r_{2020-624+1}$) and $r_{1793}$ ($r_{2020-624+397}$).
2.  Compute $s_{1397}$ and $s_{1793}$ by applying the inverse of $f$ to $r_{1397}$ and $r_{1793}$. 
3.  Compute $\tilde{s}_{2000} = g(0, s_{1397}, s_{1793})$.
4.  Compute $f(\tilde{s}_{2000})$ and use it as our guess for $r_{2000}$ with a 50% chance of success. 

After a few tries, we should be able to get the flag.

## commonfactor
---



### Challenge summary
In this challenge, we're given 4 moduli: ($N_1, N_2, N_3, N_4$). Each modulus $N_i = (p + e_i)q_i$, $p$ is 2048-bit long, each $e_i$ is 512-bit long and each $q_i$ is 1024-bit long. $p, e_i, q_i$ are unknown. To get the flag, we need to factor at least one of these moduli.



### Solution
Since $N_1 = (p+e_1)q_1, N_2 = (p+e_2)q_2$, as a consequence, $q_1N_2  - q_2N_1 = e_2q_1q_2 - e_1q_1q_2$ (keep in mind that $N_1, N_2$ are big compared to $e_1q_1q_2 - e_2q_1q_2$ (3072 bits to 2560 bits)).

Similarly, we have:

$$
\begin{cases}
    q_1N_3 - q_3N_1 = e_3q_1q_3 - e_1q_1q_3 \\
    q_1N_4 - q_4N_1 = e_4q_1q_4 - e_1q_1q_4 \\
\end{cases}
$$

Finally, we end up with this vector equation:

$$
q_1 \begin{bmatrix} 2^{1536} \\ N_2   \\ N_3  \\ N_4  \end{bmatrix} + 
q_2 \begin{bmatrix} 0        \\ -N_1  \\ 0    \\ 0    \end{bmatrix} +
q_3 \begin{bmatrix} 0        \\ 0     \\ -N_1 \\ 0    \end{bmatrix} +
q_4 \begin{bmatrix} 0        \\ 0     \\ 0    \\ -N_1 \end{bmatrix}
\text{ } = \text{ } 
\begin{bmatrix} 2^{1536}q_1 \\ e_2q_1q_2 - e_1q_1q_2 \\ e_3q_1q_3 - e_1q_1q_3 \\ e_4q_1q_4 - e_1q_1q_4 \end{bmatrix}
$$

($2^{1536}$ is there to scale $q_1$ (1024 bit) up to the size of $e_iq_1q_i - e_1q_1q_i$ (2560 bit)) 

Since the vectors on the left-hand side are known and big compared to the unknown vector on the right-hand side, we can treat them as basis vectors of a lattice and use the LLL algorithm to find the combination that produces the unknown vector. Once the unknown vector is found, we can obtain $q_1$ from its first component. Factorization of $N_1$ is now done.

## padwith2019
---



In this challenge, we're given a custom padding scheme and corresponding padding oracle which tells us if a message after being decrypted is properly padded or not. To get the flag, we need to utilize the oracle to decrypt arbitrary ciphertext blocks.

A pad in this scheme always contains one byte to encode the pad length. Other bytes of the pad are uniquely determined by its length. Therefore, padding oracle attacks like those on PKCS#7 padding scheme can be applied similarly in this case.

## yaecc
---




### Challenge summary
In this challenge, we're given a bad ECDSA/P256 implementation. According to the ECDSA algorithm, when signing, a nonce should be randomly and uniformly chosen in the range $[0, n)$, where $n$ is the order of the group of curve points. For P256, $n$ is 256-bit long. However, in the given implementation, nonces are only 240-bit long.

We're able to collect an "unlimited" number of message-signature pais. The goal is to find the private key.



### Solution
In ECDSA, the relation between a message, a private key and the signature (of that message signed by that private key) is as follows:

$$
Ra - Sk + Z = 0 \; (\text{mod } N)
$$

In which (for convenience, known values are denoted by uppercase letters):
- $(R, S)$: the signature
- $a$: the private key
- $k$: a random nonce
- $Z$: the message's hash value
- $N$: order of the group of curve points

Knowing that all $k$ are small. We can try to model the problem (finding the private key from message/signature pairs) as a CVP (closest vector problem) in lattice, then solve it. Please refer to [another write-up of mine](https://colab.research.google.com/github/nguyenduyhieukma/CTF-Writeups/blob/master/Google%20CTF%20Quals/2019/reality/reality-solution.ipynb) for an example of solving this kind of problem with SageMath.

Let $D$ be the number of message-signature pairs we have collected. Currently, we have:

$$
R_ia - S_ik_i + Z_i = 0 \; (\text{mod } N) \text{ for } i = 1,2,3,...,D
$$

Now, mutiply both sides by $S_i^{-1}$:

$$
S_i^{-1}R_ia = -S_i^{-1}Z_i + k_i \; (\text{mod } N)
$$

Let $A_i = S_i^{-1}R_i$, $B_i = -S_i^{-1}Z_i$, then:

$$
A_ia - l_iN= B_i + k_i
$$

These equations can be written equivalently as the following vector equation:

$$
a   \begin{bmatrix} A_1   \\ A_2   \\ ... \\ A_D \\ \frac{1}{2^{16}}  \end{bmatrix} \; + \;
l_1 \begin{bmatrix} -N    \\ 0     \\ ... \\ 0   \\ 0    \end{bmatrix} \; + \;
l_2 \begin{bmatrix} 0     \\ -N    \\ ... \\ 0   \\ 0    \end{bmatrix} \; + \;
...  \; + \; 
l_d \begin{bmatrix} 0     \\ 0     \\ ... \\ -N  \\ 0    \end{bmatrix}
\text{ } = \text{ } 
\begin{bmatrix}     B_1   \\ B_2   \\ ... \\ B_D \\ 0    \end{bmatrix} \; + \; 
\begin{bmatrix}     k_1   \\ k_2   \\ ... \\ k_D \\ \frac{a}{2^{16}} \end{bmatrix}
$$

($\frac{1}{2^{16}}$ is there to scale $a$ (256 bit) down to the size of $k_i$ (240 bit))

Here's our CVP:
-  The known vectors on the left-hand side are basis vectors of a lattice.
-  The known target vector is $(B_1, B_2, ..., B_D, 0)$, which we need to find the vector in the lattice closest to.
-  The unknown difference vector is $(k_1, k_2, ..., k_D, \frac{a}{2^{16}})$, which is very small compared to the basis and target vectors.

After the difference vector is found, multiplying its last component with $2^{16}$ should give us the private key.

The last question: How many message-signature pairs should we collect? Well, each message-signature pair gives us 16-bit information about the private key. Why? Because for each relation $Ra - Sk + Z = 0 \; (\text{mod } N)$, we know the 16 most significant bits of $k$ (they are all zeros), and knowdledge of $k$ is also knowdledge of $a$. Therefore, in theory, at least $\frac{256}{16} = 16$ message-signature pairs are needed. In practice, the algorithm used to extract the private key should work fine when $D$ is around 20.

## P.S.
---



To the following teams who had solved all Crypto challenges during the event: efiens (Vietnam), 0x90r00t (France), BabyPhD (Vietnam) and GatorSheavesMutably (Russia):
-   Very awesome work! I really appreciate it.

To all players:
-   Thanks for playing and hope you enjoyed the challenges. Happy New Year!